In [1]:
import pandas as pd
import numpy as np
import json
import os
import utils
import csv
import math


### Create CSV files for analysis
Note: Running this cell block will append lines to existing CSV files. Delete existing CSV files before each new run.

In [2]:
if not os.path.exists("analysis"):
    os.mkdir("analysis")


def detect_tracking(blocklist, url_list):
    """
    Check if any URLs from a list appear in a blocklist of known tracking cookies.

    Args:
        blocklist: Set of blocked domains.
        url_list: List of URLs.

    Returns:
        A list of detected trackers.
    """

    detected_trackers = []
    for url in url_list:
        if utils.get_full_domain(url) in blocklist: # FIXME: check if this is the correct way to get domain
            detected_trackers.append(url)

    return detected_trackers


def get_urls_from_har(file: str) -> list[str]:
    """
    Returns a list of cookies from an HAR file.
    [HAR Specification](http://www.softwareishard.com/blog/har-12-spec/).

    Args:
        file: Path to the HAR file.
    Returns:
        A list of cookies.
    """

    all_urls = []
    data = json.load(open(file, "r")) # parses JSON data into Python dictionary
    for entry in data["log"]["entries"]: # each entry is an HTTP request/response pair
        request = entry["request"] # extract request dictionary

        if (url := request.get("url")) and request.get("cookies"): # valid URL exists and request contains cookies
            all_urls.append(url)

    return all_urls


def get_tracking_sites(list_path: str = "inputs/blocklists/") -> set[str]:
    """
    Get tracking sites from blocklists.

    Args:
        list_path: Path to blocklists. Defaults to "inputs/blocklists/".

    Returns:
        A set of tracking sites.
    """
    lists = []
    for item in os.listdir(list_path):
        path = os.path.join(list_path, item)
        lists.append(path)

    tracking_sites = set()
    for list_path in lists:
        with open(list_path) as file:
            lines = file.readlines()
            for line in lines:
                tracking_sites.add(line.rstrip())

    # print("Tracking sites aggregated from 4 blocklists.")
    return tracking_sites


def get_directories(root: str) -> list[str]:
    """
    Return a list of directories in a given root directory.

    Args:
        root: Path to the root directory.

    Returns:
        A list of directories.
    """
    dirs = []
    for item in os.listdir(root):
        path = os.path.join(root, item)
        if os.path.isdir(path):
            dirs.append(path)

    return dirs


# Create set of tracking sites from aggregation of 4 blocklists
trackings_sites = get_tracking_sites()
# print(trackings_sites)


def analyze_har(har_path: str):
    """
    Return a list of tracking cookies detected in the specified HAR file.

    Args:
        har_path: Path to the HAR file.

    Returns:
        A list of detected tracking cookies.
    """
    urls = get_urls_from_har(har_path) # get list of URLs
    detected_list = detect_tracking(trackings_sites, urls)
    return detected_list


success_file_path = "inputs/sites/success.txt"
with open(success_file_path, "r") as success_file:
    success_lines = success_file.readlines()

# TODO: change filepath
# domain_paths = get_directories("crawls/depth1_noquery") 
domain_paths = get_directories("crawls/depth0") 

incomplete_runs = 0
total_inner_pages = 0

# Initialize two lists to store the detected trackers from normal and after_reject runs
detected_trackers_normal = []
detected_trackers_after_reject = []

for site in domain_paths:
    # Skip if site is not in success.txt
    # FIXME: success.txt currently not formatted properly; uncommenting this causes no rows to be written to CSV
    # if not any(site in line for line in success_lines):
    #     continue

    inner_site_paths = get_directories(site)
    total_inner_pages += len(inner_site_paths)

    for inner_site_path in inner_site_paths:
        normal_har_path = f"{inner_site_path}/normal.json"
        reject_har_path = f"{inner_site_path}/after_reject.json"

        if not os.path.isfile(normal_har_path) or not os.path.isfile(reject_har_path):
            # Requires both normal and intercept HAR files to exist
            incomplete_runs += 1
            continue

        detected_list_normal = analyze_har(normal_har_path)
        # print(detected_list_normal)

        # Append info into dict to turn into dataframe
        for tracker_url in detected_list_normal:
            detected_trackers_normal.append({
                "Domain": site.split("/")[2],
                "Inner Site Path": inner_site_path,
                "Tracker": tracker_url
            })

        # Create file if it doesn't exist; if it exists then write a row for each inner site path with a count of the number of trackers.
        # TODO: change filepath
        # normal_file = "analysis/depth1_noquery_normal.csv"
        normal_file = "analysis/depth0_normal.csv"
        normal_file_exists = os.path.isfile(normal_file)

        if normal_file_exists:
            with open(normal_file, mode="a", newline="") as file:
                writer = csv.writer(file)
                writer.writerow([inner_site_path, len(detected_list_normal)])
                file.flush() # bugfix where rows weren't writing: flush() clears internal buffer

        else:
            with open(normal_file, mode="w", newline="") as file:
                writer = csv.writer(file)
                writer.writerow(["Inner Site Path", "Length of Detected List"])
                writer.writerow([inner_site_path, len(detected_list_normal)])
                file.flush()


        # Repeat for files generated after run with intercept.
        detected_list_reject = analyze_har(reject_har_path)

        # Append info into dict to turn into dataframe
        for tracker_url in detected_list_reject:
            detected_trackers_after_reject.append({
                "Domain": site.split("/")[2],
                "Inner Site Path": inner_site_path,
                "Tracker": tracker_url
            })

        # TODO: change filepath
        # reject_file = "analysis/depth1_noquery_after_reject.csv"
        reject_file = "analysis/depth0_after_reject.csv"
        reject_file_exists = os.path.isfile(reject_file)

        if reject_file_exists:
            with open(reject_file, mode="a", newline="") as file:
                writer = csv.writer(file)
                writer.writerow([inner_site_path, len(detected_list_reject)])
                file.flush()
        else:
            with open(reject_file, mode="w", newline="") as file:
                writer = csv.writer(file)
                writer.writerow(["Inner Site Path", "Length of Detected List"])
                writer.writerow([inner_site_path, len(detected_list_reject)])
                file.flush()

# Create DataFrames for detected trackers in normal and after_reject crawls
# Each tracker is in a row with its domain and inner site path
df_normal = pd.DataFrame(detected_trackers_normal)
df_after_reject = pd.DataFrame(detected_trackers_after_reject)

df_normal.info()
df_after_reject.info()

['https://ad.mail.ru/static/ads-async.js', 'https://ad.mail.ru/adp/?q=1241629&pid=146&version=horizontal&vk=0&data-ad-chunk=1', 'https://ad.mail.ru/dist/vkAuth.html', 'https://xray.mail.ru/update?p=resplash&t=csp&v=1&rlog=resplash_csp&rlog_message=%7B%22documentURI%22%3A%22https%3A%2F%2Fmail.ru%2F%22%2C%22blockedURI%22%3A%22https%3A%2F%2Fcsm.va.us.criteo.net%2Fiev%3Fentry%3Dc~Idfs.Rtus.Events.FetchError~1%22%2C%22violatedDirective%22%3A%22connect-src%22%2C%22disposition%22%3A%22enforce%22%2C%22effectiveDirective%22%3A%22connect-src%22%2C%22sourceFile%22%3A%22https%3A%2F%2Fgum.criteo.com%2Fsync%3Fc%3D405%26r%3D2%26j%3Drtus_callback%22%2C%22line%22%3A1%2C%22col%22%3A26116%7D&_=1689581606066', 'https://xray.mail.ru/update?p=resplash&t=csp&v=1&i=connect-src:1&_=1689773528970', 'https://xray.mail.ru/update?p=resplash&t=csp&v=1&i=connect-src:1&_=1690347069636', 'https://xray.mail.ru/update?p=resplash&t=csp&v=1&rlog=resplash_csp&rlog_message=%7B%22documentURI%22%3A%22https%3A%2F%2Fmail.ru%2F%

In [14]:
# df_normal.tail(15)
# df_after_reject.tail(15)

In [13]:
# Group by domain
df_normal_domains = df_normal.groupby('Domain').agg(
    Num_Inner_Sites=('Inner Site Path', 'nunique'), # Count the number of unique inner site paths
    Num_Trackers_Per_Domain=('Tracker', 'count')    # Count the number of trackers for each domain
).reset_index() # makes "Domain" a column instead of an index

df_normal_domains["Average Trackers Per Page"] = df_normal_domains["Num_Trackers_Per_Domain"] / df_normal_domains["Num_Inner_Sites"]


# Group by domain
df_after_reject_domains = df_after_reject.groupby('Domain').agg(
    Num_Inner_Sites=('Inner Site Path', 'nunique'), # Count the number of unique inner site paths
    Num_Trackers_Per_Domain=('Tracker', 'count')    # Count the number of trackers for each domain
).reset_index() # makes "Domain" a column instead of an index

df_after_reject_domains["Average Trackers Per Page"] = df_after_reject_domains["Num_Trackers_Per_Domain"] / df_after_reject_domains["Num_Inner_Sites"]


# df_normal.info()
# df_normal_domains.info()
df_normal_domains.tail(15)
# df_after_reject_domains.info()
df_after_reject_domains.tail(15)
# df_after_reject.info()

,Domain,Num_Inner_Sites,Num_Trackers_Per_Domain,Average Trackers Per Page
75,sportingnews.com,1,7,7.0
76,superuser.com,1,1,1.0
77,surveymonkey.com,1,1,1.0
78,technologyreview.com,1,2,2.0
79,thomsonreuters.com,1,2,2.0
80,thoughtco.com,1,1,1.0
81,travelandleisure.com,1,1,1.0
82,tvsquared.com,1,3,3.0
83,uservoice.com,1,1,1.0
84,vertamedia.com,1,7,7.0


### Distribution of Tracking Cookies Across All Inner Pages (Regardless of Domain)
Run this cell to check that number of complete+incomplete pages equals total inner pages.

In [5]:
def compare_trackers(reject_filepath, normal_filepath):
    no_trackers_after_reject = []  # List of inner site paths with trackers in normal crawl, but no trackers after rejection
    increased_trackers = []  # List of inner site paths with more trackers after rejection than in normal crawl
    never_trackers = []  # List of inner site paths with no trackers in either normal or rejection crawl
    violating_sites = []  # List of inner site paths with trackers after we click the reject button

    with open(reject_filepath, 'r') as reject_file, open(normal_filepath, 'r') as normal_file:
        read_reject = csv.reader(reject_file)
        read_normal = csv.reader(normal_file)

        # Skip header
        next(read_reject)
        next(read_normal)

        length = 0

        # Since both csvs are sorted by inner site path, we can just iterate through both at the same time
        for normal, after_reject in zip(read_normal, read_reject):
            inner_site_path, num_trackers_normal = normal
            _, num_trackers_reject = after_reject

            if inner_site_path != _:
                raise RuntimeError("Inner site paths do not match")

            num_trackers_normal = int(num_trackers_normal)
            num_trackers_reject = int(num_trackers_reject)

            if num_trackers_normal > 0 and num_trackers_reject == 0:  # if there are trackers in normal crawl, but not after reject
                no_trackers_after_reject.append(inner_site_path)

            if num_trackers_normal < num_trackers_reject:  # if there are more trackers after reject than in normal crawl
                increased_trackers.append(inner_site_path)

            if num_trackers_normal == 0 and num_trackers_reject == 0:  # if there are no trackers in either normal or reject
                never_trackers.append(inner_site_path)

            if num_trackers_reject != 0:  # if there are trackers in reject
                violating_sites.append(inner_site_path)

            length += 1

    # from previous cell
    print("Total inner pages:", total_inner_pages)
    print("Incomplete inner pages:", incomplete_runs)
    
    print("Complete inner pages:", length)
    print("Inner pages that removed all trackers after rejection:", len(no_trackers_after_reject))
    print("Inner pages with increased trackers after rejection:", len(increased_trackers))
    print("Inner pages that never contained trackers:", len(never_trackers))
    print("Inner pages that sent cookies to 3rd party trackers after rejection:", len(violating_sites))


def get_length_detected_list(csv_reader, inner_site_path):
    for row in csv_reader:
        current_inner_site_path, length_detected_list = row
        if current_inner_site_path == inner_site_path:
            return length_detected_list

    return '0'  # If inner_site_path not found, return '0'


compare_trackers('analysis/depth0_after_reject.csv', 'analysis/depth0_normal.csv')

Total inner pages: 252
Incomplete inner pages: 2
Complete inner pages: 250
Inner pages that removed all trackers after rejection: 8
Inner pages with increased trackers after rejection: 33
Inner pages that never contained trackers: 152
Inner pages that sent cookies to 3rd party trackers after rejection: 90
